## Project outline
### Could it have been predicted?

In this project we will develop a series of safety pharmacology models using python based cheminformatic tools such as Rdkit, Scikitlearn, and pytorch. We want to see if machine learning cheminformatic toxicity models can single out the molecules that were ultimately removed from the market. I have collected a database of drugs that were withdrawn from the market due to toxicity for various reasons. This can be combined with a database of currently available medications to form a test set which can be used to test a model designed to predict toxicity. Many of the drugs that have been withdrawn over the years were withdrawn due to hepatotoxicity or DILI (drug induced liver injury) so I have collected a dataset of molecules with BSEP binding values to develop a model capable of predicting hepatoxicity. This will be the first model. If time allows, we will also gather data on other toxicities responsible for drug withdrawal such as binding the HERG (IKr) associated protein and potentially other secondary pharmacology assay targets such as Gprotein-coupledreceptors (GPCRs), enzymes, kinases, nuclear hormone receptors, ion channels and transporters.

### References
Assay Targets:

Jenkinson, S., et al., A practical guide to secondary pharmacology in drug discovery. Journal of Pharmacological and Toxicological Methods, 2020. 105.

BSEP Database:

AbdulHameed, M.D.M., R. Liu, and A. Wallqvist, Using a Graph Convolutional Neural Network Model to Identify Bile Salt Export Pump Inhibitors. ACS Omega, 2023. 8(24): p. 21853-21861.

Dataset of Withdrawn drugs:

Siramshetty, V.B., et al., WITHDRAWN--a resource for withdrawn and discontinued drugs. Nucleic Acids Res, 2016. 44(D1): p. D1080-6.

Onakpoya, I. J., Heneghan, C. J., & Aronson, J. K. (2016). Post-marketing withdrawal of 462 medicinal products because of adverse drug reactions: a systematic review of the world literature. BMC Medicine, 14, 10.

## Import Dependencies

In [2]:
import pandas as pd
import numpy as np
import pubchempy as pcp
from yellowbrick import classifier
from tqdm import tqdm
#---------------------- Therapeutic Drug Commons (TDC data) from https://tdcommons.ai/single_pred_tasks/tox/#dili-drug-induced-liver-injury
from tdc.single_pred import Tox
#---------------------- RDKit packages
from rdkit.Chem import AllChem
from molfeat import trans
#---------------------- scikit-learn packages
from sklearn import ensemble
from sklearn.model_selection import train_test_split, RandomizedSearchCV

## Data cleaning
### DILI

Define a function to convert trade & generic drug names to SMILES strings

In [4]:
def replace_drug_name_with_smiles(dataframe: pd.DataFrame, drug_name_col: str) -> pd.DataFrame:
    dataframe[drug_name_col] = dataframe[drug_name_col].map(lambda x: pcp.get_compounds(identifier=x, namespace='name')) # Get pubchem CID for each compound
    dataframe = dataframe[dataframe[drug_name_col].map(lambda d: len(d)) == 1] # Drop columns with multiple chemical identifiers
    dataframe[drug_name_col] = dataframe[drug_name_col].str[0] # Convert list of pubchempy compounds to str
    dataframe[drug_name_col] = dataframe[drug_name_col].apply(lambda x: x.isomeric_smiles) # Get isomeric smiles for pubchempy compounds
    return dataframe

Convert Xu DF from TDC library

In [28]:
tox_data = Tox(name = 'DILI')
xu_df = tox_data.get_data()

xu_df = (xu_df
        .drop('Drug_ID', axis=1)
        .rename(columns={'Drug' : 'SMILES', 'Y' : 'DILI?'})
        .astype({'DILI?' : 'Int16'})
)

xu_df.to_csv('Transformed_Data/Xu_DILI.csv')

Downloading...
100%|██████████| 26.7k/26.7k [00:00<00:00, 13.3MiB/s]
Loading...
Done!


Convert the Onakpoya DF from the withdrawn drugs list

In [ ]:
onakpoya_df = pd.read_csv('Intermediate_Data/Onakpoya_Drugs.csv', skiprows = [0]) # Read the table as csv

onakpoya_df = (onakpoya_df
    .filter(['Medicinal product', 'Reason for withdrawal']) # Drop irrelevant columns
    .replace({'‡':''}, regex=True) # Remove uninterpretable characters
)

onakpoya_df = onakpoya_df[onakpoya_df['Reason for withdrawal'].str.endswith('Liver', na = False)] # Drop non-DILI related withdrawal
onakpoya_df['Medicinal product'] = onakpoya_df['Medicinal product'].str.partition(' ')[0] # Only keep first word of drug name

onakpoya_df = replace_drug_name_with_smiles(onakpoya_df, "Medicinal product")

onakpoya_df.columns = ['SMILES', 'DILI?']

onakpoya_df = (onakpoya_df
    .filter(['DILI?', 'SMILES']) # Drop drug name
    .reindex(columns = ['SMILES', 'DILI?']) # Reorder columns
    .replace({'Liver' : 1}) # Liver = 1
)

onakpoya_df.to_csv('Transformed_Data/Onakpoya_DILI2.csv', index=False)

Convert the Livertox database, consider "A" DILI-positive, "E" DILI-negative.

In [8]:
livertox_df = pd.read_excel('Raw_Data/LiverTox_DILI.xlsx', skiprows=range(2))

values = ['A', 'E']

livertox_df = (livertox_df
    .query('`Likelihood Score` == @values')
    .filter(['Ingredient', 'Likelihood Score'])
    .rename(columns = {'Ingredient': 'drug', 'Likelihood Score': 'dili'})
    .replace({'A': 1, 'E': 0})
)

livertox_df = replace_drug_name_with_smiles(livertox_df, "drug")

print(livertox_df)

livertox_df.to_csv('Transformed_Data/Livertox_DILI.csv', index=False)

                                                   drug  dili
4            CCCS(=O)(=O)NC1CC(C1)N(C)C2=NC=NC3=C2C=CN3     0
6                                  CC(=O)NCCCS(=O)(=O)O     0
12                              CC(=O)N[C@@H](CS)C(=O)O     0
14    C1C[N+]2(CCC1[C@H](C2)OC(=O)C(C3=CC=CS3)(C4=CC...     0
15    CC1=CC=C(C=C1)/C(=C\CN2CCCC2)/C3=CC=CC(=N3)/C=...     0
...                                                 ...   ...
1493  COC(=O)[C@H]1[C@H](CC[C@@H]2[C@@H]1C[C@H]3C4=C...     0
1496     CCN(C1=CC=CC(=C1)C2=CC=NC3=C(C=NN23)C#N)C(=O)C     0
1497  CC(=O)N[C@@H]1[C@H](C=C(O[C@H]1[C@@H]([C@@H](C...     0
1501                                               [Zn]     0
1505    CC1=CC=C(C=C1)C2=C(N3C=C(C=CC3=N2)C)CC(=O)N(C)C     0

[482 rows x 2 columns]


Concatenate the data from different sources

In [ ]:
tox_df = pd.concat([xu_df, onakpoya_df])

tox_df.to_csv('Transformed_Data/Final_DILI.csv', index=False)

## BSEP inhibition

In [5]:
Hameed_df = pd.read_excel('Transformed_Data/Hameed_BSEP.xlsx', sheet_name=1, usecols = range(1, 3))

print(Hameed_df.info(2))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 925 entries, 0 to 924
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   SMILES         925 non-null    object
 1   BSEP ACTIVITY  925 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 14.6+ KB
None


# Generate Fingerprints

Define function 'generate_fingerprints'
Initialise empty list of Morgan fingerprints
for molecules in a given dataframe, generate their morgan fingerprints and append them to the dataframe
Reutrn appended dataframe as numpy array to analyse using 'shape'

Run generate_fingerprints on each molecule in the dataframe

Use shape to confirm success - First number should equal dataframe length


In [3]:
tox_df = pd.read_csv('Transformed_Data/Final_DILI.csv', index_col=0)

In [4]:
def generate_fp_column(dataframe, dataframe_smiles_col, fp_type: str) -> pd.DataFrame:
    fp_transformer = trans.MoleculeTransformer(featurizer=f'{fp_type}')
    dataframe[f"{fp_type}"] = fp_transformer.transform(dataframe_smiles_col.values)
    return dataframe

tox_df = generate_fp_column(tox_df, tox_df.SMILES, 'ecfp')

print(tox_df.info(2))

<class 'pandas.core.frame.DataFrame'>
Index: 507 entries, 0 to 456
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   SMILES  507 non-null    object
 1   DILI?   507 non-null    int64 
 2   ecfp    507 non-null    object
dtypes: int64(1), object(2)
memory usage: 15.8+ KB
None


In [5]:
morgan_df = pd.DataFrame(tox_df.iloc[:, 2])

morgan_df.insert(len(morgan_df.columns), 'DILI?', tox_df['DILI?'].astype(int)) # Insert 'DILI?' column as the last column

morgan_df.columns = morgan_df.columns.astype(str) # Set all column titles to string - Required for model

print(morgan_df.info(2))

#X = np.array(morgan_df['M3FP'])
X = morgan_df.iloc[:, 0] # Features
y = morgan_df.iloc[:, 1] # Labels

<class 'pandas.core.frame.DataFrame'>
Index: 507 entries, 0 to 456
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ecfp    507 non-null    object
 1   DILI?   507 non-null    int32 
dtypes: int32(1), object(1)
memory usage: 9.9+ KB
None


In [8]:
# Set up scoring methods for hyperparameter tuning
scoring = ['r2', 'neg_root_mean_squared_error']

# Enable multithreading functionality
import multiprocessing
n_jobs = multiprocessing.cpu_count()-1

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.75)

In [9]:
# Instantiate Random Forest Regressor
model_rf = ensemble.RandomForestClassifier(random_state=42)

# Instantiate a dict of paramaters of RFR
params_rf = {
            'bootstrap': [True, False],
            'max_depth': [range(1, 4096, 1000), None],
            'max_features': ['auto', 'sqrt', 'log2', 1],
            'min_samples_leaf': range(1, 8, 1),
            'min_samples_split': range(1, 8, 1),
            'n_estimators': range(10, 800, 10)
            }

In [11]:
# Run RandomizedSearchCV to optimise random forest hyperparameters
rf_cv = RandomizedSearchCV(model_rf, params_rf, n_iter=64, n_jobs=n_jobs, random_state=42, scoring='r2') #5-fold precedended in AbdulHameed

rf_cv.fit(list(X_train),y_train)

# Print scores
print('Tuned Logistic Regression Parameters: {}'.format(rf_cv.best_params_)) 
print('Best score is {}'.format(rf_cv.best_score_))

Tuned Logistic Regression Parameters: {'n_estimators': 680, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 1, 'max_depth': None, 'bootstrap': True}
Best score is 0.19498257839721264


In [ ]:

classifier.confusion_matrix(rf_cv, list(X_test), y_test, cmap="Greens")

classifier.roc_auc(rf_cv, list(X_test), y_test)

In [ ]:
from sklearn import svm

model_svm = svm.LinearSVC(random_state=42)

model_svm.fit(list(X_train), y_train)

classifier.confusion_matrix(model_svm, list(X_test), y_test, cmap="Greens")
classifier.roc_auc(model_svm, list(X_test), y_test, binary=True)

In [ ]:
from tpot import TPOTRegressor

pipeline_optimiser = TPOTRegressor(random_state = 42, n_jobs=n_jobs)

pipeline_optimiser.fit(X_train, y_train)
print(pipeline_optimiser.score(X_test, y_test))

In [ ]:
# Instantiate an XGBoost

model_gb = ensemble.GradientBoostingClassifier(random_state=42)

params_gb = {
            'max_depth': [2, 3, 5, 10, 15],
            'learning_rate': [0.05, 0.1, 0.15, 0.20]
            }

from sklearn.model_selection import RandomizedSearchCV

gb_cv = RandomizedSearchCV(model_gb, params_gb, cv = 5, n_iter=32, n_jobs=n_jobs, random_state=42, scoring='r2')

gb_cv.fit(list(X_train), y_train)

# Print scores
print('Tuned Logistic Regression Parameters: {}'.format(gb_cv.best_params_)) 
print('Best score is {}'.format(gb_cv.best_score_))

In [ ]:
classifier.confusion_matrix(gb_cv, list(X_test), y_test, cmap="Greens")
classifier.roc_auc(gb_cv, list(X_test), y_test)

## Thanks To

https://www.youtube.com/watch?v=-oHqQBUyrQ0

https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

https://github.com/PatWalters/practical_cheminformatics_posts/blob/main/solubility/literature_solubility_model.ipynb

https://leftwinglow.github.io/BachelorsProject/

https://github.com/gashawmg/Molecular-fingerprints/blob/main/Calculating%20molecular%20fingerprints%20available%20in%20RDkit%20.ipynb

https://github.com/gashawmg/Avalon-fingerprints-for-machine-learning/blob/main/Avalon%20fingerprints%20for%20predictive%20modeling.ipynb